In [1]:
import pandas as pd
import numpy as np
import pickle

import sys
sys.path.insert(0, "../src")
sys.path.insert(0, "../")
import shutil
from d2v_recommender import *
from config import config

from autosklearn.classification import AutoSklearnClassifier
from autosklearn.metrics import roc_auc
import lightgbm
import sklearn

In [2]:
d2v = D2V_Recommender()
data_dict = d2v.load_data_dict(config.data_dict_path)
X_train = data_dict["X_train"].reshape((-1,1))
X_test = data_dict["X_test"].reshape((-1,1))
y_train = data_dict["y_train"]
y_test = data_dict["y_test"]

print(data_dict["X_train"].shape, data_dict["y_train"].shape, data_dict["X_test"].shape, data_dict["y_test"].shape)

(181277,) (181277,) (1640385,) (1640385,)


In [4]:
pure_train_size = int(0.1 * len(X_train)) 
train_data = lightgbm.Dataset(X_train[:pure_train_size, :], label=y_train[:pure_train_size])
val_data = lightgbm.Dataset(X_train[pure_train_size:, :], label=y_train[pure_train_size:])

In [5]:
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 1
}

model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=val_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

[LightGBM] [Warning] objective is set=binary, application=binary will be ignored. Current value: objective=binary
[LightGBM] [Warning] objective is set=binary, application=binary will be ignored. Current value: objective=binary
[LightGBM] [Info] Number of positive: 4370, number of negative: 13757
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001871 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 18127, number of used features: 1
[LightGBM] [Warning] objective is set=binary, application=binary will be ignored. Current value: objective=binary
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.241077 -> initscore=-1.146785
[LightGBM] [Info] Start training from score -1.146785
[1]	valid_0's auc: 0.526312
Training until validation scores don't improve for 100 rounds
[2]	valid_

In [ ]:
# # Auto-ML

# shutil.rmtree(config.output_automl_path, ignore_errors=True)
# shutil.rmtree(config.tmp_automl_path, ignore_errors=True)
# automl = AutoSklearnClassifier(
#     time_left_for_this_task=180,
#     per_run_time_limit=60,
#     tmp_folder=config.tmp_automl_path,  # automated
#     output_folder=config.output_automl_path,
#     # metric=roc_auc
# )

# automl.fit(X_train, 
#             y_train,
#             dataset_name='d2v_small')

# ############################################################################
# # Print the final ensemble constructed by auto-sklearn
# # ====================================================

# print(automl.show_models())

In [10]:
from sklearn.metrics import *

###########################################################################
# Get the Score of the final ensemble
# ===================================
probs = model.predict(X_test)
predictions = probs > 0.5
# proba = model.predict_proba(X_test)

print("Accuracy score:", accuracy_score(y_test, predictions))
print("MCC:", matthews_corrcoef(y_test, predictions))
# print("AUC:", auc(y_test, predictions))

Accuracy score: 0.7549800808956434
MCC: 0.0


In [9]:
cm = confusion_matrix(y_test, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=["Ignored","Matched"])
disp.plot()

ValueError: Classification metrics can't handle a mix of binary and continuous targets